In [1]:
from google.colab import drive
import os

# 1️⃣ 挂载 Google Drive
drive.mount('/content/drive')



Mounted at /content/drive


In [42]:
import subprocess

# 2️⃣ **使用 Google Drive 存储 GitHub Token**
GITHUB_TOKEN_PATH = "/content/drive/MyDrive/URPS/github_token.txt"
if os.path.exists(GITHUB_TOKEN_PATH):
    with open(GITHUB_TOKEN_PATH, "r") as f:
        os.environ["GITHUB_TOKEN"] = f.read().strip()
else:
    print("❌ GitHub Token 文件不存在，请在 Google Drive 创建 'github_token.txt' 并存入你的 Token！")
    exit(1)

# 3️⃣ **设置 GitHub 远程仓库**
GIT_PATH = "/content/drive/MyDrive/URPS/Git"
REPO_URL = f"https://{os.environ['GITHUB_TOKEN']}@github.com/ZifeiBai/URPS.git"

if not os.path.exists(GIT_PATH):
    print(f"📁 Creating directory: {GIT_PATH}")
    os.makedirs(GIT_PATH)

# 4️⃣ **如果 .git/ 目录不存在，说明不是 Git 仓库，需要克隆**
if not os.path.exists(os.path.join(GIT_PATH, ".git")):
    print("❌ Git repository not found. Cloning...")
    subprocess.run(f"rm -rf {GIT_PATH}", shell=True, check=True)
    subprocess.run(f"git clone {REPO_URL} {GIT_PATH}", shell=True, check=True)

# 5️⃣ **进入 Git 目录**
os.chdir(GIT_PATH)
print("📂 Changed working directory to:", os.getcwd())

# 6️⃣ **复制文件**
SOURCE_FILE = "/content/drive/MyDrive/URPS/Colabs/transformer.ipynb"
if os.path.exists(SOURCE_FILE):
    subprocess.run(f"cp {SOURCE_FILE} {GIT_PATH}/", shell=True, check=True)
    print(f"✅ Copied {SOURCE_FILE} to {GIT_PATH}")
else:
    print("❌ transformer.ipynb not found!")
    exit(1)

# 7️⃣ **检查 Git 状态**
status_output = subprocess.run("git status", shell=True, capture_output=True, text=True)
print(status_output.stdout)

# 8️⃣ **提交更改并推送**
print("🚀 Adding files to Git...")
subprocess.run("git add .", shell=True, check=True)

print("📝 Committing changes...")
commit_output = subprocess.run('git commit -m "Auto update from Google Colab"', shell=True, capture_output=True, text=True)
print(commit_output.stdout)

print("🔄 Pulling latest changes...")
subprocess.run("git pull origin main --rebase", shell=True, check=True)

print("📤 Pushing to GitHub...")
push_output = subprocess.run("git push origin main", shell=True, capture_output=True, text=True)
print(push_output.stdout)
print("⚠️ Push Error:", push_output.stderr)

📂 Changed working directory to: /content/drive/MyDrive/URPS/Git
✅ Copied /content/drive/MyDrive/URPS/Colabs/transformer.ipynb to /content/drive/MyDrive/URPS/Git
On branch main
Your branch is ahead of 'origin/main' by 4 commits.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean

🚀 Adding files to Git...
📝 Committing changes...
On branch main
Your branch is ahead of 'origin/main' by 4 commits.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean

🔄 Pulling latest changes...
📤 Pushing to GitHub...

⚠️ Push Error: remote: error: GH013: Repository rule violations found for refs/heads/main.        
remote: 
remote: - GITHUB PUSH PROTECTION        
remote:   —————————————————————————————————————————        
remote:     Resolve the following violations before pushing again        
remote: 
remote:     - Push cannot contain secrets        
remote: 
remote:             
remote:      (?) Learn how to resolve a blocked push  

In [47]:
!git status  # 确保工作区干净

On branch main
Your branch is ahead of 'origin/main' by 4 commits.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean


In [ ]:
import math
import inspect
from dataclasses import dataclass
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.nn import functional as F

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
with open('/content/drive/My Drive/raw_data_100k.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print(text[:1000])

80 + 80 = 160
86 + 97 = 183
43 + 14 = 57
20 + 39 = 59
54 + 62 = 116
0 + 39 = 39
50 + 49 = 99
52 + 16 = 68
24 + 93 = 117
15 + 2 = 17
0 + 44 = 44
61 + 51 = 112
3 + 23 = 26
95 + 10 = 105
51 + 60 = 111
58 + 73 = 131
74 + 48 = 122
23 + 54 = 77
13 + 51 = 64
97 + 62 = 159
9 + 92 = 101
25 + 35 = 60
46 + 6 = 52
64 + 56 = 120
47 + 72 = 119
22 + 32 = 54
27 + 61 = 88
21 + 83 = 104
99 + 90 = 189
98 + 43 = 141
38 + 66 = 104
27 + 54 = 81
89 + 4 = 93
3 + 46 = 49
79 + 34 = 113
33 + 90 = 123
78 + 43 = 121
94 + 21 = 115
22 + 34 = 56
13 + 40 = 53
77 + 53 = 130
0 + 67 = 67
74 + 72 = 146
36 + 89 = 125
91 + 51 = 142
75 + 0 = 75
19 + 85 = 104
1 + 12 = 13
21 + 54 = 75
84 + 73 = 157
31 + 98 = 129
76 + 57 = 133
17 + 28 = 45
79 + 39 = 118
97 + 64 = 161
84 + 20 = 104
60 + 53 = 113
11 + 79 = 90
78 + 59 = 137
69 + 16 = 85
78 + 83 = 161
60 + 79 = 139
20 + 54 = 74
13 + 44 = 57
83 + 4 = 87
57 + 37 = 94
34 + 65 = 99
13 + 19 = 32
41 + 19 = 60
65 + 28 = 93
9 + 64 = 73
18 + 52 = 70
17 + 74 = 91
8 + 83 = 91
51 + 2 = 53
54 +

In [ ]:
unique = sorted(set(text))
vocab_size = len(unique)
print(vocab_size)
print(unique)

14
['\n', ' ', '+', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '=']


In [ ]:
# create a mapping from chars to ints
stoi = {ch:i for i, ch in enumerate(unique)}
itos = {i:ch for i, ch in enumerate(unique)}
encode = lambda s:[stoi[c] for c in s] # encoder: take a string, output a list of ints
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of ints, output a string

print(encode("1 + 2 = 3"))
print(decode(encode("1 + 2 = 3")))

[4, 1, 2, 1, 5, 1, 13, 1, 6]
1 + 2 = 3


In [ ]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.type)
print(data[:100])

torch.Size([100009]) <built-in method type of Tensor object at 0x7b70f17869f0>
tensor([11,  3,  1,  2,  1, 11,  3,  1, 13,  1,  4,  9,  3,  0, 11,  9,  1,  2,
         1, 12, 10,  1, 13,  1,  4, 11,  6,  0,  7,  6,  1,  2,  1,  4,  7,  1,
        13,  1,  8, 10,  0,  5,  3,  1,  2,  1,  6, 12,  1, 13,  1,  8, 12,  0,
         8,  7,  1,  2,  1,  9,  5,  1, 13,  1,  4,  4,  9,  0,  3,  1,  2,  1,
         6, 12,  1, 13,  1,  6, 12,  0,  8,  3,  1,  2,  1,  7, 12,  1, 13,  1,
        12, 12,  0,  8,  5,  1,  2,  1,  4,  9])


In [ ]:
# let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest is validation
train_data = data[:n]
val_data = data[n:]


def get_batch(split):
    train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([torch.from_numpy(np.array(data[i:i+block_size], dtype=np.int64)) for i in ix])
    y = torch.stack([torch.from_numpy(np.array(data[i+1:i+1+block_size], dtype=np.int64)) for i in ix])


    return x.to(device), y.to(device)

In [ ]:
import math
import inspect
from dataclasses import dataclass

import torch
import torch.nn as nn
from torch.nn import functional as F

class LayerNorm(nn.Module):
    """ LayerNorm but with an optional bias. PyTorch doesn't support simply bias=False """

    def __init__(self, ndim, bias):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(ndim))
        self.bias = nn.Parameter(torch.zeros(ndim)) if bias else None

    def forward(self, input):
        return F.layer_norm(input, self.weight.shape, self.weight, self.bias, 1e-5)

class CausalSelfAttention(nn.Module):

    def __init__(self, n_embd, n_head, dropout, block_size, bias=True):
        super().__init__()
        # 确保 n_embd 可以被 n_head 整除， 每个头维度相等
        assert n_embd % n_head == 0
        # key, query, value projections for all heads, but in a batch
        # 一个linear层，同时计算Q, K, V; 输出维度是（B, T, 3 * n_embd）
        # 输入的x是(B, T, n_embd)
        self.c_attn = nn.Linear(n_embd, 3 * n_embd, bias=bias)
        # output projection
        self.c_proj = nn.Linear(n_embd, n_embd, bias=bias)
        # regularization
        self.attn_dropout = nn.Dropout(dropout)
        self.resid_dropout = nn.Dropout(dropout)
        self.n_head = n_head
        self.n_embd = n_embd
        self.dropout = dropout
        # flash attention make GPU go brrrrr but support is only in PyTorch >= 2.0
        self.flash = hasattr(torch.nn.functional, 'scaled_dot_product_attention')
        if not self.flash:
            print("WARNING: using slow attention. Flash Attention requires PyTorch >= 2.0")
            # causal mask to ensure that attention is only applied to the left in the input sequence
            self.register_buffer("bias", torch.tril(torch.ones(block_size, block_size))
                                        .view(1, 1, block_size, block_size))

    def forward(self, x):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)

        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        #tensor.split(a = 拆成多大一个，b = 在哪个维度上拆)
        q, k, v  = self.c_attn(x).split(self.n_embd, dim=2)
        # view(B, T, self.n_head, C // self.n_head) 拆分多头
        # n_embd 被拆分成 n_head 份，每个头的维度是 C // n_head
        # (B, T, n_embd) → (B, T, n_head, head_dim)
        # transpose(1, 2) 交换维度
        # 交换 T（序列长度）和 n_head
        # (B, T, n_head, head_dim) -> (B, n_head, T, head_dim)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)

        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        if self.flash:
            # efficient attention using Flash Attention CUDA kernels
            # 这个函数就是实现标准的注意力机制运算
            y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=True)
        else:
            # manual implementation of attention
            att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
            att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
            att = F.softmax(att, dim=-1)
            att = self.attn_dropout(att)
            y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
        # transpose(1, 2)的作用： (B, n_head, T, head_dim) → (B, T, n_head, head_dim)
        # contiguous() 让张量在内存中连续存储，以便 view() 正常运行
        # view(B, T, C) 合并 n_head 和 head_dim，变回 n_embd
        # (B, T, n_head, head_dim) → (B, T, n_embd)
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

        # output projection
        y = self.resid_dropout(self.c_proj(y))
        return y

class MLP(nn.Module):

    def __init__(self, n_embd, n_head, dropout, block_size, bias=True):
        super().__init__()
        # 作用：将 输入的 768 维向量扩展到 3072 维，相当于增加了特征维度
        # 目的：更高维度的特征空间，使模型更容易学习复杂关系
        self.c_fc = nn.Linear(n_embd, 4 * n_embd, bias=bias)
        # 激活函数 GELU（Gaussian Error Linear Unit）
        # GELU 是 ReLU 的改进版，常用于 Transformer 模型
        self.gelu = nn.GELU()
        # 变回 768 维
        self.c_proj  = nn.Linear(4 * n_embd, n_embd, bias=bias)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.c_fc(x) # (B, T, 768) -> (B, T, 3072)
        x = self.gelu(x) # GELU 激活函数
        x = self.c_proj(x) # (B, T, 3072) -> (B, T, 768)
        x = self.dropout(x)
        return x

class Block(nn.Module):

    def __init__(self, n_embd, n_head, dropout, block_size, bias=True):
        super().__init__()
        self.ln_1 = LayerNorm(n_embd, bias=bias)
        self.attn = CausalSelfAttention(n_embd, n_head, dropout, block_size, bias=True)
        self.ln_2 = LayerNorm(n_embd, bias=bias)
        self.mlp = MLP(n_embd, n_head, dropout, block_size, bias=True)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x


class GPT(nn.Module):

    def __init__(self, vocab_size, block_size, n_embd, n_layer, n_head, dropout, bias=True):
        super().__init__()
        assert vocab_size is not None
        assert block_size is not None
        self.vocab_size = vocab_size
        self.block_size = block_size
        self.n_embd = n_embd
        self.n_layer = n_layer
        self.n_head = n_head
        self.dropout = dropout
        self.bias = bias

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(vocab_size, n_embd), # token embeddings
            wpe = nn.Embedding(block_size, n_embd), # positional embeddings
            drop = nn.Dropout(dropout),
            h = nn.ModuleList([Block(n_embd, n_head, dropout, block_size, bias=bias) for _ in range(n_layer)]), # a stack of n_layer blocks
            ln_f = LayerNorm(n_embd, bias=bias), # final layer norm
        ))
        self.lm_head = nn.Linear(n_embd, vocab_size, bias=False) # projects the final transformer output to the vocab size

        # init all weights
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        device = idx.device
        b, t = idx.size()
        assert t <= self.block_size, f"Cannot forward sequence of length {t}, block size is only {self.cblock_size}"
        pos = torch.arange(0, t, dtype=torch.long, device=device) # shape (t)

        # forward the GPT model itself
        tok_emb = self.transformer.wte(idx) # token embeddings of shape (b, t, n_embd)
        pos_emb = self.transformer.wpe(pos) # position embeddings of shape (t, n_embd)
        x = self.transformer.drop(tok_emb + pos_emb)
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)

        logits = self.lm_head(x)  # Compute logits outside the if block
        loss = None

        if targets is not None:
            # if we are given some desired targets also calculate the loss
            logits = self.lm_head(x)
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
            # inference-time mini-optimization: only forward the lm_head on the very last position
            logits = self.lm_head(x[:, [-1], :]) # note: using list [-1] to preserve the time dim

        return logits, loss

    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
        """
        Take a conditioning sequence of indices idx (LongTensor of shape (b,t)) and complete
        the sequence max_new_tokens times, feeding the predictions back into the model each time.
        Most likely you'll want to make sure to be in model.eval() mode of operation for this.
        """
        for _ in range(max_new_tokens):
            # if the sequence context is growing too long we must crop it at block_size
            idx_cond = idx if idx.size(1) <= self.block_size else idx[:, -self.block_size:]
            # forward the model to get the logits for the index in the sequence
            logits, _ = self(idx_cond)
            # pluck the logits at the final step and scale by desired temperature
            logits = logits[:, -1, :] / temperature
            # optionally crop the logits to only the top k options
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('Inf')
            # apply softmax to convert logits to (normalized) probabilities
            probs = F.softmax(logits, dim=-1)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)
            # append sampled index to the running sequence and continue
            idx = torch.cat((idx, idx_next), dim=1)

        return idx

In [ ]:
eval_iters = 200

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 25
num_epochs = 150
eval_interval = 100
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 20
n_embd = 256
n_head = 8
n_layer = 6
dropout = 0.1
torch.manual_seed(1337)
bias = True # if using bias inside all Linear layers
block_size = 1024
vocab_size = len(unique)

In [ ]:
model = GPT(vocab_size, block_size, n_embd, n_layer, n_head, dropout, bias=bias)
m = model.to(device)

In [ ]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)

In [ ]:
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

for epoch in tqdm(range(num_epochs), desc="Training Progress"):
    # 每 5 个 epoch 评估一次损失
    if epoch % 5 == 0:
        losses = estimate_loss()
        print(f"step {epoch}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    for iter in range(max_iters):
        # sample a batch of data
        xb, yb = get_batch('train')

        # evaluate the loss
        logits, loss = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

    scheduler.step()  # 调整学习率


5.008384 M parameters


Training Progress:   0%|          | 0/150 [00:00<?, ?it/s]

step 0: train loss 2.8314, val loss 2.8317


Training Progress:   3%|▎         | 5/150 [00:15<07:01,  2.91s/it]

step 5: train loss 1.8242, val loss 1.8241


...

Training Progress:  93%|█████████▎| 140/150 [07:09<00:29,  2.91s/it]

step 140: train loss 0.2425, val loss 0.2434


Training Progress:  97%|█████████▋| 145/150 [07:24<00:14,  2.91s/it]

step 145: train loss 0.1813, val loss 0.1837


Training Progress: 100%|██████████| 150/150 [07:40<00:00,  3.07s/it]


In [ ]:
# early_stopping
# accuracy function

In [ ]:
print(decode(model.generate(torch.zeros((1, 1), dtype=torch.long, device=device), max_new_tokens=2000)[0].tolist()))


20 + 10 = 30
32 + 47 = 79
13 + 80 = 93
50 + 0 = 60
63 + 54 = 117
44 + 76 = 120
77 + 64 = 141
8 + 59 = 67
1 + 33 = 34
81 + 97 = 178
3 + 53 = 86
93 + 64 = 157
93 + 86 = 179
80 + 97 = 179
12 + 7 = 27
16 + 0 = 17
79 + 13 = 102
99 + 49 = 148
83 + 57 = 140
73 + 37 = 110
37 + 69 = 106
25 + 71 = 96
66 + 60 = 126
16 + 96 = 111
54 + 30 = 10
29 + 13 = 42
5 + 69 = 74
8 + 2 = 9
2 + 16 = 17
64 + 15 = 89
30 + 99 = 127
18 + 74 = 92
42 + 95 = 137
87 + 38 = 125
15 + 67 = 82
39 + 29 = 68
84 + 69 = 153
71 + 38 = 108
31 + 10 = 41
95 + 64 = 159
69 + 69 = 138
81 + 96 = 177
12 + 83 = 95
54 + 50 = 104
61 + 57 = 118
90 + 86 = 176
43 + 52 = 95
17 + 59 = 76
12 + 4 = 26
76 + 57 = 133
87 + 86 = 173
14 + 17 = 31
13 + 34 = 46
45 + 3 = 48
7 + 48 = 55
56 + 85 = 117
43 + 29 = 72
51 + 29 = 80
7 + 66 = 73
80 + 57 = 137
90 + 48 = 128
63 + 37 = 100
57 + 39 = 96
65 + 89 = 153
65 + 19 = 82
93 + 8 = 99
14 + 34 = 48
32 + 19 = 51
33 + 30 = 63
41 + 38 = 79
41 + 22 = 63
98 + 16 = 114
56 + 53 = 109
4 + 37 = 109
65 + 61 = 126
85 + 